In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 20.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [ ]:
import os
os._exit(00)

In [ ]:
!pip install datasets
!pip install tokenizers
!pip install transformers
!pip install seqeval
!pip install wandb
# !pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 765.7 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=086296dbfa493c6ee07f3115245d9b24531f2faeb1bf8b49c9418c4d9b03f33e
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.2 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
train_data = load_dataset("json", data_files = "/content/drive/MyDrive/SABDA-NER/trainData.json")

validation_data = load_dataset("json", data_files = "/content/drive/MyDrive/SABDA-NER/validationData.json")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
label_names = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

# Assuming we have label_names defined somewhere
def tokenize_adjust_labels(all_samples_per_split):
    tokenized_samples = tokenizer.batch_encode_plus(all_samples_per_split["tokens"], is_split_into_words=True)
    total_adjusted_labels = []

    for k in range(len(tokenized_samples["input_ids"])):
        word_ids_list = tokenized_samples.word_ids(batch_index=k)
        existing_label_ids = all_samples_per_split["ner_tags"][k]
        i = 0
        adjusted_label_ids = []
        prev_wid = None

        for wid in word_ids_list:
            if wid is None:
                adjusted_label_ids.append(-100)
            elif wid != prev_wid:
                if i < len(existing_label_ids):
                    adjusted_label_ids.append(existing_label_ids[i])
                    prev_wid = wid
                    i += 1
                else:
                    # Handling the case where existing_label_ids has fewer elements
                    adjusted_label_ids.append(-100)
            else:
                adjusted_label_ids.append(existing_label_ids[i])

        total_adjusted_labels.append(adjusted_label_ids)

    tokenized_samples["labels"] = total_adjusted_labels
    return tokenized_samples

tokenized_dataset = train_data.map(tokenize_adjust_labels, batched=True)


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Map:   0%|          | 0/15999 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset["train"]

print(next(iter(tokenized_dataset['train'])))
#one hot encoding: converting token into numerical representation
toks=[101, 887, 11549, 33270, 92344, 11986, 58011, 18438, 24562, 41013, 13466, 851, 51230, 11714, 10826, 38518, 95579, 10914, 851, 51230, 26876, 117, 851, 31222, 18321, 11554, 49069, 891, 881, 69334, 11208, 18351, 17110, 78954, 18479, 898, 13841, 111202, 13466, 32087, 887, 51140, 22949, 38063, 29161, 13718, 884, 111193, 65261, 20321, 40340, 920, 102]
tt=['बर्मेली', 'भाषा', 'मात्रै', 'जान्ने', 'अनिल', 'की', 'श्रीमती', 'अनि', 'नेपाली', ',', 'अंग्रे', 'जी', 'र', 'थोडाबहुत', 'हिन्दी', 'समझने', 'लेकिन', 'बोल्न', 'उस्तो', 'नआउने', 'हामी', '।']

{'ner_tags': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'tokens': ['बर्मेली', 'भाषा', 'मात्रै', 'जान्ने', 'अनिल', 'की', 'श्रीमती', 'अनि', 'नेपाली', ',', 'अंग्रे', 'जी', 'र', 'थोडाबहुत', 'हिन्दी', 'समझने', 'लेकिन', 'बोल्न', 'उस्तो', 'नआउने', 'हामी', '।'], 'input_ids': [101, 887, 11549, 33270, 92344, 11986, 58011, 18438, 24562, 41013, 13466, 851, 51230, 11714, 10826, 38518, 95579, 10914, 851, 51230, 26876, 117, 851, 31222, 18321, 11554, 49069, 891, 881, 69334, 11208, 18351, 17110, 78954, 18479, 898, 13841, 111202, 13466, 32087, 887, 51140, 22949, 38063, 29161, 13718, 884, 111193, 65261, 20321, 40340, 920, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 

In [ ]:
tokenized_dataset_validation=validation_data.map(tokenize_adjust_labels, batched=True)

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
# for tracking our machine learning work we used wandb.

import os
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="SABDA-NER",

    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "bert",
    "dataset": "SABDA-NER_data",
    "epochs": 7,
    }
)

# simulate training
epochs = 7
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset

    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})

# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


acc,▁▃▆▇█
loss,█▆▂▁▂
acc,0.89836
loss,0.18259


In [ ]:
os.environ["WANDB_API_KEY"]="332e7a429154c53329127ba60cf2dddf123e701d"
os.environ["WANDB_ENTITY"]="backstar"
os.environ["WANDB_PROJECT"]="SABDA-NER"

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
import numpy as np
from datasets import load_metric
metric = load_metric("seqeval") #(seqeval)Sequence Labeling Evaluation
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    flattened_results = {
        "overall_precision": results["overall_precision"],
        "overall_recall": results["overall_recall"],
        "overall_f1": results["overall_f1"],
        "overall_accuracy": results["overall_accuracy"],
    }
    for k in results.keys():
      if(k not in flattened_results.keys()):
        flattened_results[k+"_f1"]=results[k]["f1"]

    return flattened_results

In [ ]:
model = AutoModelForTokenClassification.from_pretrained("bert-base-multilingual-cased", num_labels=len(label_names))

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import accelerate
accelerate.__version__

'0.25.0'

In [ ]:

training_args = TrainingArguments(
    output_dir="./SABDA-NER",
    evaluation_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=7,
    weight_decay=0.01,
    logging_steps = 1000,
    report_to="wandb",
    run_name="grateful-bird-3",
    save_strategy='no'
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset_validation["validate"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
wandb.finish()


wandb: Currently logged in as: anupabde17 (backstar). Use `wandb login --relogin` to force relogin


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Overall Precision,Overall Recall,Overall F1,Overall Accuracy,Loc F1,Org F1,Per F1
1000,0.138400,0.050314,0.882710,0.894191,0.888413,0.985394,0.847752,0.841636,0.926775
2000,0.056300,0.025012,0.932093,0.955075,0.943444,0.993070,0.912787,0.920803,0.968791
3000,0.034900,0.018899,0.945162,0.969598,0.957224,0.994587,0.936240,0.940877,0.974823
4000,0.022600,0.010172,0.968706,0.985557,0.977059,0.997106,0.971117,0.973603,0.981481
5000,0.015200,0.005933,0.980913,0.992419,0.986633,0.998452,0.982959,0.983726,0.989703
6000,0.010100,0.003858,0.987401,0.994335,0.990856,0.998932,0.989231,0.988447,0.992675
7000,0.007100,0.003461,0.989843,0.995372,0.992600,0.999109,0.991648,0.989373,0.994424


eval/LOC_f1,▁▄▅▇███
eval/ORG_f1,▁▅▆▇███
eval/PER_f1,▁▅▆▇███
eval/loss,█▄▃▂▁▁▁
eval/overall_accuracy,▁▅▆▇███
eval/overall_f1,▁▅▆▇███
eval/overall_precision,▁▄▅▇▇██
eval/overall_recall,▁▅▆▇███
eval/runtime,▁▃▂▇▃▄█
eval/samples_per_second,█▆▆▂▆▅▁
eval/steps_per_second,█▆▆▂▆▅▁


In [ ]:
# # Save the trained model
# model_checkpoint = "/content/drive/MyDrive/SABDA-NER/"  # Choose a directory to save your model
# trainer.save_model(model_checkpoint)

In [5]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer

# Load the trained model and tokenizer
model_checkpoint = "/content/drive/MyDrive/SABDA-NER/"  # Use the path where you saved your trained model
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Define the NER pipeline using the trained model
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)

# Test sentence
# sentence = "अनुप को घर स्याङ्जा हो।"

# sentence = "अनुप को घर स्याङ्गजा हो।"
# sentence = "पीएम नरेंद्र मोदी ने गुरुवार को तीन आपराधिक न्याय विधेयकों (Criminal Justice Bills) के पारित होने की सराहना की. नए कानून देश में औपनिवेशिक युग में बनाए गए कानूनों की जगह लेंगे. पीएम मोदी ने कहा कि यह सार्वजनिक सेवा और कल्याण पर केंद्रित कानूनों के साथ एक नए युग की शुरुआत है."
# sentence = "राम कृष्ण सहकारी सँस्था लिमीटेड।"
# sentence = "राम कृष्ण ।"
# sentence = "राम कृष्ण श्रेठ।"
# sentence = "कृष्ण श्रेठ।"
# sentence = "सिता र कृष्ण श्रेठ।"

# sentence = """
#  राम ले काठमाडौँ बिश्वोबिद्यालयमा पढ्छ।

# """
# sentence = """ शिभम सिमेन्ट """
sentence = """
पूर्वको सिमान्त सहर मेचीनगर नगरपालिका-६ का सामान्य पृष्ठभूमिमा तीन आमा धनकुमारी राई, रायमा बेगम र टीकादेवी श्रेष्ठबीच एउटै समानता छ । त्यो हो, उनीहरूले अभाव र संघर्षको पृष्ठभूमि अनि छोरीहरूको रहर र सपनालाई उकेरा लगाउँदै राष्ट्रिय क्रिकेट खेलाडी बनाउँदाको सफलताको कथा ।

"""


# Perform NER on the test sentence
ner_results = ner_pipeline(sentence)

# Initialize variables
current_word = ""
combined_entities = []

# Iterate through NER results
for result in ner_results:
    # Check if the token has the '##' prefix
    if result['word'].startswith('##'):
        # Concatenate the token to the current word
        current_word += result['word'][2:]  # Remove the '##' prefix
    else:
        # If it's a new word, add the previous word (if any) to the list
        if current_word:
            combined_entities.append({'word': current_word, 'entity': current_entity, 'score': current_score})
        # Set the current word to the current token
        current_word = result['word']
        current_entity = result['entity']
        current_score = result['score']

# Add the last word (if any) to the list
if current_word:
    combined_entities.append({'word': current_word, 'entity': current_entity, 'score': current_score})

# Display the combined entities
for result in combined_entities:
    print(f"Entity: {result['word']}, Label: {result['entity']}, Score: {result['score']}")


Entity: पूर्वको, Label: LABEL_0, Score: 0.9998961687088013
Entity: सिमान्त, Label: LABEL_0, Score: 0.9999244213104248
Entity: सहर, Label: LABEL_0, Score: 0.999872088432312
Entity: मेचीनगर, Label: LABEL_3, Score: 0.9991023540496826
Entity: नगरपालिका, Label: LABEL_4, Score: 0.9993699193000793
Entity: -, Label: LABEL_0, Score: 0.9999217987060547
Entity: ६, Label: LABEL_0, Score: 0.999955415725708
Entity: का, Label: LABEL_0, Score: 0.9999665021896362
Entity: सामान्य, Label: LABEL_0, Score: 0.9999667406082153
Entity: पृष्ठभूमिमा, Label: LABEL_0, Score: 0.9999699592590332
Entity: तीन, Label: LABEL_0, Score: 0.9999662637710571
Entity: आमा, Label: LABEL_0, Score: 0.9999555349349976
Entity: धनकुमारी, Label: LABEL_1, Score: 0.9994812607765198
Entity: राई, Label: LABEL_2, Score: 0.9993231296539307
Entity: ,, Label: LABEL_0, Score: 0.9998868703842163
Entity: रायमा, Label: LABEL_1, Score: 0.9995276927947998
Entity: बेगम, Label: LABEL_2, Score: 0.9994584918022156
Entity: र, Label: LABEL_0, Score: 0.